In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import os
import numpy as np


In [7]:
def load_images_with_labels(root_folder):
    images_with_labels = []
    for object_folder in os.listdir(root_folder):
        object_path = os.path.join(root_folder, object_folder)
        if os.path.isdir(object_path):
            for filename in os.listdir(object_path):
                img_path = os.path.join(object_path, filename)
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        label = object_folder
                        images_with_labels.append((img, label))
    return images_with_labels

# Define root folder containing subfolders for each object category
current_directory = os.getcwd()
root_folder = os.path.dirname(current_directory) + '/data/Garbage_classification/Garbage classification'

# Load images with labels
images_with_labels = load_images_with_labels(root_folder)


[(array([[[220, 232, 236],
          [220, 232, 236],
          [220, 232, 236],
          ...,
          [187, 202, 211],
          [187, 202, 211],
          [187, 202, 211]],
  
         [[220, 232, 236],
          [220, 232, 236],
          [220, 232, 236],
          ...,
          [187, 202, 211],
          [187, 202, 211],
          [187, 202, 211]],
  
         [[220, 232, 236],
          [220, 232, 236],
          [220, 232, 236],
          ...,
          [187, 202, 211],
          [187, 202, 211],
          [187, 202, 211]],
  
         ...,
  
         [[156, 121, 125],
          [160, 125, 129],
          [158, 123, 127],
          ...,
          [103,  64,  66],
          [108,  69,  71],
          [113,  74,  76]],
  
         [[167, 132, 136],
          [168, 133, 137],
          [162, 127, 131],
          ...,
          [101,  62,  64],
          [106,  67,  69],
          [111,  72,  74]],
  
         [[167, 132, 136],
          [167, 132, 136],
          [160, 125, 129

In [10]:
def preprocess_image(img, target_size):
    # Resize image
    img_resized = cv2.resize(img, target_size)

    # Normalize pixel values
    img_normalized = img_resized / 255.0

    return img_normalized

# Define target size for resizing
target_size = (224, 224)

# Preprocess each image in the dataset
preprocessed_data = []
for img, label in images_with_labels:
    preprocessed_img = preprocess_image(img, target_size)
    preprocessed_data.append((preprocessed_img, label))

# Convert preprocessed_data to NumPy arrays
X = np.array([data[0] for data in preprocessed_data])
y = np.array([data[1] for data in preprocessed_data])
X


array([[[[0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         ...,
         [0.74117647, 0.8       , 0.83137255],
         [0.7372549 , 0.79607843, 0.83137255],
         [0.73333333, 0.79215686, 0.82745098]],

        [[0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         ...,
         [0.7372549 , 0.79607843, 0.83137255],
         [0.73333333, 0.79215686, 0.82745098],
         [0.73333333, 0.79215686, 0.82745098]],

        [[0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         [0.8627451 , 0.90980392, 0.9254902 ],
         ...,
         [0.73333333, 0.79215686, 0.82745098],
         [0.73333333, 0.79215686, 0.82745098],
         [0.72941176, 0.78823529, 0.82352941]],

        ...,

        [[0.56470588, 0.42745098, 0.44313725],
         [0.63137255, 0.49411765, 0.50980392]

In [ ]:
# Define the CNN model
def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (224, 224, 3)
num_classes = len(set(y))

# Create the model
model = create_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print(f"Test Accuracy: {accuracy}")
